In [1]:
from models.training import GMFTrainer
from models.base_model import GMF
from evaluation.evaluator import Evaluator
import numpy as np
import pandas as pd
import torch

In [2]:
sessions_train_df = pd.read_json('data_files/train_sessions.jsonl', lines=True)
tracks_df = pd.read_json('data_files/tracks.jsonl', lines=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset = torch.utils.data.TensorDataset(
    torch.LongTensor(sessions_train_df['user_id'].values).to(device),
    torch.LongTensor(sessions_train_df['track_id'].values).to(device),
    torch.FloatTensor(sessions_train_df['score'].values).to(device)
)

train_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=128,
    shuffle=True
)

unique_users_count = sessions_train_df['user_id'].nunique()
unique_tracks_count = len(tracks_df)

In [3]:
gmf = GMF(unique_users_count, unique_tracks_count, 64)
gmf_trainer = GMFTrainer(gmf, train_loader, device)

In [4]:
gmf_trainer.train(10, 0.01)

Epoch 1/10 - Average Loss: 0.1429
Epoch 2/10 - Average Loss: 0.1374
Epoch 3/10 - Average Loss: 0.1374


KeyboardInterrupt: 

In [5]:
ground_truth_df = pd.read_json('data_files/val_sessions.jsonl', lines=True)
ground_truth_df = ground_truth_df[ground_truth_df['user_id'] < 1000]
all_users = torch.arange(1000).to(device)
all_items = torch.arange(unique_tracks_count).to(device)
all_pairs = torch.cartesian_prod(all_users, all_items)
user_input = all_pairs[:,0]
item_input = all_pairs[:,1]

In [6]:
predictions = []
batch_size = 10000
gmf.eval()
for i in range(0, len(all_pairs), batch_size):
    batch_pairs = all_pairs[i:i + batch_size]
    batch_user_input = batch_pairs[:,0].to(device)
    batch_item_input = batch_pairs[:,1].to(device)
    
    with torch.no_grad():
        batch_predictions = gmf.forward(batch_user_input, batch_item_input)
        predictions.append(batch_predictions)

all_predictions = torch.cat(predictions)

In [7]:
all_pairs_np = all_pairs.cpu().numpy()
all_predictions_np = all_predictions.cpu().numpy()

recommendations_df = pd.DataFrame({
    'user_id': all_pairs_np[:, 0],
    'track_id': all_pairs_np[:, 1],
    'score': all_predictions_np
})

In [12]:
evaluator2 = Evaluator([5, 10, 20, 30, 50])

In [13]:
metrics = evaluator2.evaluate(recommendations_df, ground_truth_df)

In [16]:
for metric, value in metrics.items():
    print(f'{metric}: {value * 100}%')

NDCG@5: 99.91292107971039%
NDCG@5_std: 0.7648140548486799%
Recall@5: 0.013821483852649002%
Recall@5_std: 0.13974309497832507%
Precision@5: 0.24000000000000002%
Precision@5_std: 2.1787948841382816%
NDCG@10: 99.87302027922004%
NDCG@10_std: 0.8376597539450039%
Recall@10: 0.025048542436235558%
Recall@10_std: 0.19274965189888474%
Precision@10: 0.2%
Precision@10_std: 1.4007005254378833%
NDCG@20: 99.77673698250403%
NDCG@20_std: 1.0113474037145722%
Recall@20: 0.07874948654134359%
Recall@20_std: 0.5134336692260161%
Precision@20: 0.24000000000000002%
Precision@20_std: 1.2222495219446372%
NDCG@30: 99.70846496692879%
NDCG@30_std: 1.043247938605575%
Recall@30: 0.10852740199433146%
Recall@30_std: 0.5496050154785117%
Precision@30: 0.24333333333333335%
Precision@30_std: 0.964680458060565%
NDCG@50: 99.58641218142547%
NDCG@50_std: 1.0834232173818297%
Recall@50: 0.20811692031841036%
Recall@50_std: 0.7154922025828906%
Precision@50: 0.27999999999999997%
Precision@50_std: 0.8162513499322781%
